In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv("../../data/cleaned/out.csv")

2023-05-01 11:28:48.693248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 11:28:50.062681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yonosoysantiago/miniconda3/envs/tf211_conda/lib/:/home/yonosoysantiago/miniconda3/envs/tf211_conda/lib/
2023-05-01 11:28:50.062899: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yonosoysanti

In [2]:
# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
word_index = tokenizer.word_index

max_sequence_length = max(len(seq) for seq in sequences)
input_data = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode the labels
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(data['label'])
labels = to_categorical(integer_encoded_labels, num_classes=7)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size=0.2, random_state=42)

In [4]:
embedding_dim = 100
vocab_size = len(word_index) + 1

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(7, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-05-01 11:28:53.515242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:28:53.515431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:28:53.545187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:28:53.545377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:28:53.545523: I tensorflow/compiler/xla/stream_executo

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [8]:
num_epochs = 10
batch_size = 4096

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=num_epochs,
    batch_size=batch_size,
    verbose=2
)

Epoch 1/10
11/11 - 3s - loss: 0.5660 - accuracy: 0.8176 - val_loss: 1.0939 - val_accuracy: 0.6596 - 3s/epoch - 254ms/step
Epoch 2/10
11/11 - 2s - loss: 0.5374 - accuracy: 0.8280 - val_loss: 1.0896 - val_accuracy: 0.6571 - 2s/epoch - 217ms/step
Epoch 3/10
11/11 - 3s - loss: 0.5137 - accuracy: 0.8362 - val_loss: 1.1127 - val_accuracy: 0.6556 - 3s/epoch - 228ms/step
Epoch 4/10
11/11 - 3s - loss: 0.4848 - accuracy: 0.8459 - val_loss: 1.1342 - val_accuracy: 0.6523 - 3s/epoch - 231ms/step
Epoch 5/10
11/11 - 3s - loss: 0.4617 - accuracy: 0.8526 - val_loss: 1.1496 - val_accuracy: 0.6498 - 3s/epoch - 238ms/step
Epoch 6/10
11/11 - 2s - loss: 0.4411 - accuracy: 0.8590 - val_loss: 1.2215 - val_accuracy: 0.6445 - 2s/epoch - 218ms/step
Epoch 7/10
11/11 - 2s - loss: 0.4237 - accuracy: 0.8641 - val_loss: 1.1753 - val_accuracy: 0.6465 - 2s/epoch - 220ms/step
Epoch 8/10
11/11 - 2s - loss: 0.4087 - accuracy: 0.8697 - val_loss: 1.2335 - val_accuracy: 0.6447 - 2s/epoch - 217ms/step
Epoch 9/10
11/11 - 3s - 

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy}')